# Spark Wrangle Exercises
This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.<br>
>A. Read the `case`, `department`, and `source` data into their own spark dataframes.<br>
B. Write the code necessary to store the source data in both __csv__ and __json__ format, store these as `sources_csv` and `sources_json`.<br>
C. Inspect your _folder_ structure. What do you notice?<br>
D. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.<br>

In [1]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StringType, StringType

In [2]:
# Create a spark session object to build spark dataframes.
spark = SparkSession.builder.getOrCreate()

In [44]:
# 1. Read the case, dept, and source datasets into spark dataframes.
df_case = spark.read.csv('data/case.csv', header=True, sep=',', inferSchema=True)
df_dept = spark.read.csv('data/dept.csv', header=True, sep=',', inferSchema=True)
df_source = spark.read.csv('data/source.csv', header=True, sep=',', inferSchema=True)

## case.csv

In [4]:
# The dimensions of the case dataset
df_case.count(), len(df_case.columns)

(841704, 14)

In [5]:
# Efficient way to show the number of non-null values in a spark dataframe
df_case.summary().show(1, truncate=False, vertical=True)

-RECORD 0----------------------
 summary              | count  
 case_id              | 841704 
 case_opened_date     | 841704 
 case_closed_date     | 823594 
 SLA_due_date         | 841671 
 case_late            | 841704 
 num_days_late        | 841671 
 case_closed          | 841704 
 dept_division        | 841704 
 service_request_type | 841704 
 SLA_days             | 841671 
 case_status          | 841704 
 source_id            | 841704 
 request_address      | 841704 
 council_district     | 841704 
only showing top 1 row



In [6]:
# Display the column names
df_case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [7]:
# Display a single record from the case dataset to understand what an obersation represents.
# The `case` dataset has many columns that wrap around when displayed.
# Pass: truncate=False, vertical=True to display the full values of each column.
df_case.show(2, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

### `case` dataset: Record 0 takeaway
Each row represents a 311 call.
- Unique case_id
- Case opened and closed datetimes.
    - Date and time represents the total time it took the department to complete the case: Paper work, travel, tasks, etc.
- The SLA due date is set 999 days from the case open date.
- This type of case is late if it exceeds 999 days.
- The number of days late is a negative number starting at -999, representing the case open date
    - The fraction missing means that the case was opened and closed within half of a day.
- Department division identifies the city department assigned to the case.
- Service type request indentifies the type of request issued to the department.
- SLA days is the maximum number of days this case can be opened.
- Case status identifies if a case is opened or closed.
- Source id may represent the division/personnel generating the request.
- Request address is the location requesting 311 service.
- Council district represents the district a location resides in.
    - Council representatives can find common 311 requests among their constituents.

## dept.csv

In [8]:
# The dimensions of the dept dataset
df_dept.count(), len(df_dept.columns)

(39, 4)

In [9]:
# Display the column names and datatypes
# Column data types are correct
df_dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

## source.csv

In [10]:
# The dimensions of the source dataset
df_source.count(), len(df_source.columns)

(140, 2)

In [11]:
# Display the column names and datatypes
# Column data types are correct
df_source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

Save the source dataset in csv and JSON format.
```python
df_source.write.json('data/source_json', mode='overwrite')
df_source.write.csv('data/source_csv', mode='overwrite')
```
This saves each sparkdataframe as a folder with 2 files:
- _SUCCESS
- A partition csv file.

# Data Preparation

In [45]:
df_case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [46]:
# Rename technical jargon into laymans terms...
df_case = df_case.withColumnRenamed("SLA_due_date", "case_due_date")

In [47]:
# Drop missing values from the dataframe
df_case = df_case.na.drop()

In [49]:
# Create a variable to store a datetime string format
dt_format = 'M/d/yy H:mm'

# Cast each date column from string datatypes into datetime datatypes
df_case = (
    df_case.withColumn('case_opened_date', to_timestamp('case_opened_date', dt_format))
           .withColumn('case_closed_date', to_timestamp('case_closed_date', dt_format))
           .withColumn('case_due_date', to_timestamp('case_due_date', dt_format))
)
                       

In [50]:
# Cast columns with binary outcomes from string datatype to boolean datatype.
df_case = (
    df_case.withColumn('case_closed', when(col('case_closed') == 'Open', True).otherwise(False))
           .withColumn('case_late', when(col('case_late') == 'YES', True).otherwise(False))
)

In [52]:
# Transform the negative values to 0. This means that the case wasn't late.
# If there is a non-zero value in this column, it means that the case was late x number of days.
df_case = (
    df_case.withColumn('num_days_late', when(col('num_days_late') < 0, 0).otherwise(col('num_days_late')))
)

In [62]:
# Create new columns to measure the status of open v. closed requests.
(
df_case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
    .withColumn("days_to_closed", datediff("case_closed_date", "case_opened_date"))
    .withColumn("case_lifetime", when(col('case_closed') == False, col("case_age")).otherwise(col("days_to_closed")))
    ).show(2, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | 0.0                                  
 case_closed          | false                                
 dept_division        | field operations                     
 service_request_type | stray animal                         
 SLA_days             | 999.0                                
 case_status          | closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 5                                    
 case_age             | 1064                                 
 days_to

In [53]:
# Normalize string datatype columns
df_case = (
    df_case.withColumn('case_status', lower(trim('case_status')))
           .withColumn('dept_division', lower(trim('dept_division')))
           .withColumn('service_request_type', lower(trim('service_request_type')))
           .withColumn('request_address', lower(trim('request_address')))
)

In [54]:
df_case.show(1, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | 0.0                                  
 case_closed          | false                                
 dept_division        | field operations                     
 service_request_type | stray animal                         
 SLA_days             | 999.0                                
 case_status          | closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?<br>
> This question refers to the `case` dataset.

In [38]:
print('The latest open service request')
df_case.filter(col('case_closed') == False).agg(max('num_days_late')).show()

The latest open service request
+------------------+
|max(num_days_late)|
+------------------+
|       519.6980787|
+------------------+



## 2. How many Stray Animal cases are there?

In [ ]:
print('The latest open service request')
df_case.filter(col('case_closed') == False).agg(max('num_days_late')).show()

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

## 4. Convert the council_district column to a string column.

## 5. Extract the year from the case_closed_date column.

## 6. Convert num_days_late from days to hours in new columns num_hours_late.

## 7. Join the case data with the source and department data.

## 8. Are there any cases that do not have a request source?

## 9. What are the top 10 service request types in terms of number of requests?

In [20]:
df_case.groupby('service_request_type').count().sort(col('count').desc()).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|no pickup                       |86538|
|overgrown yard/trash            |63665|
|bandit signs                    |32312|
|damaged cart                    |30233|
|front or side yard parking      |28367|
|stray animal                    |26745|
|aggressive animal(non-critical) |24758|
|cart exchange request           |21855|
|junk vehicle on private property|20531|
|pot hole repair                 |20344|
+--------------------------------+-----+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

## 11. Does number of days late depend on department?

## 12. How do number of days late depend on department and request type?